In [1]:
import torch
import torchvision
from models import DeepLabV3
from datasets import gta5dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torchvision import transforms as T
from torchvision.utils import make_grid
import torch.nn as nn
import numpy as np
from torch.utils.tensorboard import SummaryWriter
import time
import os
from tqdm.notebook import tqdm
from utils import colorize_mask
from PIL import Image

In [2]:
dataset = gta5dataset("./datas/")
# Data
dataloader = DataLoader(dataset,
                                  shuffle=True,
                                  batch_size=4,
                                  num_workers=0,
                                  pin_memory=False)

device = torch.device("cuda:0")
model = DeepLabV3().to(device)      

epochs = 31
# train loop
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr = 0.01, momentum = 0.9 , weight_decay = 1e-4)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.1 )

# loss
def compute_loss(output,target):
    ce = nn.CrossEntropyLoss(ignore_index=255)
    ce_loss = ce(output,target)
    return ce_loss

# output dir
run_time = time.strftime("%Y-%m-%d_%H-%M-%S",time.localtime())
output_dir = "./train-runs"
output_dir = os.path.join(output_dir,run_time)
if os.path.exists(output_dir):
    os.makedir(output_dir)
# Tensorboard
writer = SummaryWriter(os.path.join(output_dir,"log"))

# visualize
def visualize(im,gt,pred):
    vis_transform = T.Compose([T.ToTensor()])
    
    im = im[0].data.cpu()
    im = np.array(im.permute(1,2,0),dtype=np.uint8)

    im = vis_transform(im)
    
    gt = gt[0].data.cpu().numpy()
    gt = colorize_mask(gt).convert('RGB')
    gt = vis_transform(gt)

    pred = pred[0].data.max(0)[1].cpu().numpy()
    pred = colorize_mask(pred).convert('RGB')
    pred = vis_transform(pred)

    grid = torch.stack([im,gt,pred],0)
    grid = make_grid(grid.cpu(), nrow=3, padding=5)
    return grid

d:\Anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
d:\Anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [3]:
# Train loop

interval = 500
for epoch in range(0,epochs):
    # one eopch  
    model.train()
    print(f"epoch {epoch} starts at {time.strftime(r'%Y-%m-%d_%H-%M-%S',time.localtime())}")
    for i,(index,image,label) in tqdm(enumerate(dataloader)):

        output = model(image.to(device))['out']

        loss = compute_loss(output,label.to(device))
        # optimization step:
        optimizer.zero_grad() # (reset gradients)
        loss.backward() # (compute gradients)
        optimizer.step() # (perform optimization step)


        step = epoch*len(dataloader)+i 
        if step % interval ==0:
            writer.add_scalar("total_loss",loss,step)   
            grid = visualize(image,label,output)
            writer.add_image(f'img_gt_pred', grid, step)

    # validate: visualize one image 
    # model.eval()
    # save checkpoint
    lr_scheduler.step()
    
    if epoch % 10 == 0:
        if not os.path.exists(os.path.join(output_dir,"checkpoint")):
            os.mkdir(os.path.join(output_dir,"checkpoint"))
        torch.save(model.state_dict(),
                    os.path.join(output_dir,"checkpoint",f"deeplabv3-{epoch}.pth"))


epoch 0 starts at 2022-10-01_21-58-45


[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
 255]



d:\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


epoch 1 starts at 2022-10-01_21-58-46


[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  18 255]

epoch 2 starts at 2022-10-01_21-58-46


...

[  0   1   2   3   4   5   6   7   8   9  10  11  13  14 255]

epoch 27 starts at 2022-10-01_21-58-48


[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  17 255]

epoch 28 starts at 2022-10-01_21-58-48


[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  17 255]

epoch 29 starts at 2022-10-01_21-58-48


[  0   1   2   3   4   5   6   7   8   9  10  11  13  14  15 255]

epoch 30 starts at 2022-10-01_21-58-48


[  0   1   2   3   4   5   6   7   8   9  10  11  13  14  15 255]

